In [119]:
import oil_sele, catch_api
import pandas as pd
from selenium import webdriver as wd
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
import requests
import datetime 
from bs4 import BeautifulSoup
import time
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db, auth, firestore
from datetime import datetime as dt

In [120]:
str_current_date = datetime.date.today().isoformat()
current_date = pd.to_datetime(str_current_date)
# today = int(current_date.strftime('%Y%m%d'))
today = '20230413'
# print(type(current_date))
data_nan_date= []
print(current_date)
print(today)
 


2023-04-13 00:00:00
20230413


In [121]:
def make_catch(date):
    catch = catch_api.api_day(date,'고등어')
    if catch.isnull().values.any()==True:
        print(catch['일시'])
        catch = catch.rename(columns={'위판일자':'일시'})
        catch['일시'] = pd.to_datetime(catch['일시'])
    else:
        catch = catch.reset_index()
        catch = catch.rename(columns={'위판일자':'일시'})
        catch['일시'] = pd.to_datetime(catch['일시'])
    return catch

In [122]:
def make_oil(date):
    oil = oil_sele.oilCrawling()
    oil_df = pd.DataFrame([[str(date),oil]],columns=['일시','WTI'])
    oil_df['일시'] = pd.to_datetime(oil_df['일시'])
    return oil_df

In [123]:
def past_oil(date):
    oil = oil_sele.past_Crawling(date)
    oil_df = pd.DataFrame([[str(date),oil]],columns=['일시','WTI'])
    oil_df['일시'] = pd.to_datetime(oil_df['일시'])
    return oil_df

In [124]:
def loop_nan_date(datelist):
    time.sleep(3600)
    for i in datelist:
        catch = make_catch(i)
    return catch


In [125]:
def make_rawData(date):
    oil = make_oil(date)
    catch = make_catch(date)
    # if catch.isnull().values.any()==True :
    #     catch_nan_date.append(catch['일시'].dt.strftime('%Y%m%d').tolist())
    rawData = catch.merge(oil,on='일시',how='inner')
    rawData['일시'] = pd.to_datetime(rawData['일시'])
    rawData = rawData.set_index('일시')
    return rawData

In [126]:
rawData = make_rawData(today)
if rawData.isnull().values.any() == True :
    rawData = rawData.reset_index()
    data_nan_date.append(rawData['일시'].dt.strftime('%Y%m%d').tolist())
    pass
# elif rawData.isnull().values.any()==False :
#     #firebase에 올리기

error code : 03
error message : 조회된 데이터가 없습니다.


In [127]:
while data_nan_date:
    for i in data_nan_date :
        print(i)
        catch = make_catch(i)
        oil = past_oil(i)
        rawData = catch.merge(oil,on='일시',how='inner')
        rawData['일시'] = pd.to_datetime(rawData['일시'])
        rawData = rawData.set_index('일시')
        if rawData.isnull().values.any()==False:
            data_nan_date.remove(i)
        else : pass


['20230413']
error code : 03
error message : 조회된 데이터가 없습니다.


ParserError: Unknown string format: ['20230413']

In [ ]:
rawData

품종    위판중량    WTI
일시                            
2023-04-09  고등어  8106.5  83.20

In [ ]:
# if rawData.isnull().values.any()==False:
#     cred = credentials.Certificate('findsea-77ce8-firebase-adminsdk-dkvm6-74ecc418c0.json')

#     app = firebase_admin.initialize_app(cred)
#     db = firestore.client()
#     rawData.columns = ['date','catch(kg)','ws(mPs)','atm(hPa)','humid(%)','tmp(C)','wTmp(C)','waveH(m)','waveDH(m)','wP(sec)','wti']
#     rawData['date'] = pd.to_datetime(rawData['date'])
#     rawData['date']=rawData['date'].dt.strftime('%Y%m%d')
#     rawData['date']=rawData['date'].astype(int)

In [ ]:
# for year in range(2013, 2024):
#   year_data = rawData['date'] <= ((year * 10000) + 1231)
#   for month in range(1, 13) :
#     month_data = rawData['date'] <= ((((year * 10000) + (month * 100)) + 31))
#     for day in range(1, 32):
#       day_data = rawData['date'] == (((year*10000) + (month * 100) + day))
#       day_data_df = rawData[day_data]
#       if day_data_df.empty==True:
#         continue
#       # day_data_df['createdTime'] = datetime.timestamp()
#       if month < 10 :
#         doc_ref = db.collection(u'fishes').document(u'scombers').collection(u'raw').  document(u'south').collection(str(year)).document('0'+str((month*100)+day).zfill(2))
#       else: 
#         doc_ref = db.collection(u'fishes').document(u'scombers').collection(u'raw').document(u'south').collection(str(year)).document(str((month*100)+day).zfill(2))
#       doc_ref.set({
#           u'date': day_data_df['date'].tolist()[0],
#           u'catch(kg)': day_data_df['catch(kg)'].tolist()[0],
#           u'wti': day_data_df[u'wti'].tolist()[0],
#           u'ws(mPs)': day_data_df[u'ws(mPs)'].tolist()[0],
#           u'atm(hPa)': day_data_df[u'atm(hPa)'].tolist()[0],
#           u'humid(%)': day_data_df[u'humid(%)'].tolist()[0],
#           u'tmp(C)': day_data_df[u'tmp(C)'].tolist()[0],
#           u'wTmp(C)': day_data_df[u'wTmp(C)'].tolist()[0],
#           u'waveH(m)': day_data_df[u'waveH(m)'].tolist()[0],
#           u'waveDH(m)': day_data_df[u'waveDH(m)'].tolist()[0],
#           u'wP(sec)': day_data_df[u'wP(sec)'].tolist()[0],
#           u'createdTime': firestore.SERVER_TIMESTAMP
#     }) 이 부분 로직 생각해보기